In [20]:
using JuMP, HiGHS, DataFrames, PATHSolver

In [2]:
generator_DF = DataFrame(
    name = ["A","B","C","D","E"],
    cost = [30, 20, 40, 0, 46],
    capacity = [20,20,60,30,5],
    rate = [1, 0.5, 1.3, 0, 0.45],
    zone = [1,1,2,2,1]
)
colnames = names(generator_DF)#filter(x -> x != "name", names(generator_DF))
gen_data = Dict()
for colname in colnames
    gen_data[colname] = JuMP.Containers.DenseAxisArray(generator_DF[!,colname], generator_DF[!,"name"])
end

g_name = generator_DF.name
demand = [51,49]

2-element Vector{Int64}:
 51
 49

In [18]:
model = Model(SCS.Optimizer)
@variables(model, begin
    gen[g_name] >= 0
    flow[g_name] >= 0
    virtual_gen[g_name, zone=1:2] >= 0
end)

@constraints(model, begin
    gen_cap[g in g_name], gen[g] <= gen_data["capacity"][g]
    gen_flow[g in g_name], flow[g] <= gen[g]
    export_priority[g in g_name], flow[g] <= flow["D"]
    demand_balance[zone in [1,2]], sum(virtual_gen[g,zone] for g in g_name)  == demand[zone]
    #demand_balance2, sum(gen[g] for g in g_name) == sum(demand[zone] for zone in [1,2])
    # NO BCA; ADD GEN COST
end)

for generator in g_name
    for zone in [1,2]
        if gen_data["zone"][generator] == zone
            @constraint(model, virtual_gen[generator, zone] == gen[generator]-flow[generator])
        else
            @constraint(model, virtual_gen[generator, zone] == flow[generator])
        end
    end
end

@objective(model, Min, sum(gen_data["cost"][g]*gen[g] for g in g_name) + sum(gen_data["rate"][g]*virtual_gen[g,1]*0 for g in g_name if gen_data["zone"][g] == 1))

optimize!(model)

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 20, constraints m: 47
cones: 	  z: primal zero / dual free vars: 12
	  l: linear vars: 35
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 78, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.68e+002 3.56e+001 8.41e+003 1.30e+003 1.00e-001 2.07e-004 
   150|1.44e-003 6.83e-005 6.32e-002 2.20e+003 1.00e-001 4.42e-004 
--------------------------------

In [4]:
value.(virtual_gen)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["A", "B", "C", "D", "E"]
    Dimension 2, Base.OneTo(2)
And data, a 5×2 Matrix{Float64}:
 20.0  -0.0
  1.0  19.0
 -0.0  30.0
 30.0   0.0
 -0.0  -0.0

In [37]:
using Ipopt, Complementarity, HiGHS
CAPEX = [1000000; 200]
cf = [0.5 0.5 0.5 ;0.3 0.3 0.3]


KKTModel = Model(Ipopt.Optimizer)
@variables(KKTModel, begin
    gen[i in 1:2, t in 1:3] >=0
    cap[i in 1:2] >= 0
    nse[t in 1:3] >= 0
    λg[i in 1:2, t in 1:3] >= 0
    λc[t in 1:3]
end)

for i in 1:2
    @complements(KKTModel, 0 <= CAPEX[i]-λg[i,1]*cf[i,1]-λg[i,2]*cf[i,2]-λg[i,3]*cf[i,3] , cap[i]>=0)
    for t in 1:3
        @complements(KKTModel, 0 <= 10*cf[i,t]+λg[i,t]-λc[t], gen[i,t] >= 0)
        @complements(KKTModel, 0<= -gen[i,t]+cap[i]*cf[i,t], λg[i,t] >=0)
    end
end

for t in 1:3
    @complements(KKTModel, 0<= 1000-λc[t], nse[t]>=0)
end

@constraints(KKTModel, begin
    demandBalance[t in 1:3], sum(gen[i,t] for i in 1:2) + nse[t] == 1000
end)


optimize!(KKTModel)


This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:       59
Number of nonzeros in inequality constraint Jacobian.:       50
Number of nonzeros in Lagrangian Hessian.............:      101

Total number of variables............................:       20
                     variables with only lower bounds:       17
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       20
Total number of inequality constraints...............:       34
        inequality constraints with only lower bounds:       34
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.00e+03 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [38]:
value.(cap)

2-element Vector{Float64}:
    1.412040284186234e-7
 3333.3333330981227